In [ ]:
#Path: TensorFlow/Tejas/
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
import keras.utils
from keras import utils as np_utils
from keras.models import Sequential
from keras.layers import  Dense, Dropout, Activation, Flatten
from librosa.core import stft,istft
import soundfile as sf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
import random
from IPython.display import Audio
%matplotlib inline

Found GPU at: /device:GPU:0


In [ ]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.9.2


In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.9851128430000244
GPU (s):
0.037329595999835874
GPU speedup over CPU: 79x


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install librosa 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import librosa

##Load Data

In [ ]:
path_clean = '/content/drive/MyDrive/ENGR-E-533 /DATA/HW2/train_clean_male.wav'
s, sr=librosa.load(path_clean, sr=None) 
print("The sampling rate(sr) for clean is {}".format(sr))
S=librosa.stft(s, n_fft=1024, hop_length=512)
path_dirty = '/content/drive/MyDrive/ENGR-E-533 /DATA/HW2/train_dirty_male.wav'
sn, sr=librosa.load(path_dirty, sr=None)
print("The sampling rate(sr) for dirty is {}".format(sr))
X=librosa.stft(sn, n_fft=1024, hop_length=512)

print(" The shape of S is {}".format(S.shape))
print(" The shape of X is {}".format(X.shape))



S_abs = np.abs(S).T
X_abs = np.abs(X).T

X_abs=np.concatenate((np.random.rand(19,513)*1e-20,X_abs))

print(" The shape of |S| is {}".format(S_abs.shape))
print(" The shape of |X| is {}".format(X_abs.shape))


The sampling rate(sr) for clean is 16000
The sampling rate(sr) for dirty is 16000
 The shape of S is (513, 2459)
 The shape of X is (513, 2459)
 The shape of |S| is (2459, 513)
 The shape of |X| is (2478, 513)


In [ ]:
image_X = np.zeros([X_abs.shape[0]-19,20,X_abs.shape[1]])
for i in range(X_abs.shape[0]-19):
  image_X[i,:,:]=X_abs[i:i+20,:]
print(image_X.shape)


'''
image_y = np.zeros([S.shape[0]-19,20,S.shape[1]])
for i in range(S.shape[0]-19):
  image_ytrain[i,:,:]=S[i:i+20,:]
image_ytrain.shape'''



(2459, 20, 513)


'\nimage_y = np.zeros([S.shape[0]-19,20,S.shape[1]])\nfor i in range(S.shape[0]-19):\n  image_ytrain[i,:,:]=S[i:i+20,:]\nimage_ytrain.shape'

In [ ]:
print(image_X.shape)


(2459, 20, 513)


In [ ]:
image_X = np.expand_dims(image_X, axis = 1)

In [ ]:
print(image_X.shape)

(2459, 1, 20, 513)


In [ ]:
model = tf.keras.Sequential([
    
    tf.keras.layers.Conv2D(filters = 64, kernel_size = (5,5),padding = 'same',activation='relu',input_shape=(1,20,513)),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (1,1),padding='same'),
    tf.keras.layers.Conv2D(filters=128, kernel_size = (2,2),padding='same',activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (3,3), strides = (1,1),padding='same'),
    #tf.keras.layers.Conv2D(filters=64, kernel_size = (3,3),padding='same',activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1024,activation = "relu"),
    tf.keras.layers.Dense(units=513,activation = "relu")
  ])

model.compile(optimizer='adam',loss='mse')

In [ ]:
model.fit(image_X, S_abs, batch_size=128,epochs=100,verbose=True)

Epoch 1/100
20/20 [==============================] - 1s 15ms/step - loss: 0.0753
Epoch 2/100
20/20 [==============================] - 0s 10ms/step - loss: 0.0558
Epoch 3/100
20/20 [==============================] - 0s 10ms/step - loss: 0.0424
Epoch 4/100
20/20 [==============================] - 0s 10ms/step - loss: 0.0329
Epoch 5/100
20/20 [==============================] - 0s 10ms/step - loss: 0.0271
Epoch 6/100
20/20 [==============================] - 0s 10ms/step - loss: 0.0230
Epoch 7/100
20/20 [==============================] - 0s 10ms/step - loss: 0.0203
Epoch 8/100
20/20 [==============================] - 0s 10ms/step - loss: 0.0180
Epoch 9/100
20/20 [==============================] - 0s 10ms/step - loss: 0.0167
Epoch 10/100
20/20 [==============================] - 0s 10ms/step - loss: 0.0152
Epoch 11/100
20/20 [==============================] - 0s 10ms/step - loss: 0.0134
Epoch 12/100
20/20 [==============================] - 0s 10ms/step - loss: 0.0116
Epoch 13/100
20/20 [=====

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_78 (Conv2D)          (None, 1, 20, 64)         820864    
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 1, 20, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_79 (Conv2D)          (None, 1, 20, 128)        32896     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 1, 20, 128)       0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 2560)              0         
                                                                 
 dense_7 (Dense)             (None, 1024)             

In [ ]:
S_pred = model.predict(image_X)

77/77 [==============================] - 0s 3ms/step


In [ ]:
S_pred.shape

(2459, 513)

In [ ]:
S_pred = np.multiply(X/np.abs(X),np.abs(S_pred))

In [ ]:
S_pred.shape

(513, 2459)

In [ ]:
import math

In [ ]:
s_t = istft(S_abs.T, hop_length=512)
sh_t = istft(S_pred, hop_length=512)

num = np.dot(s_t, s_t)
den = np.dot((s_t - sh_t),(s_t - sh_t))
SNR = -10*math.log10(np.divide(num,den))
print(SNR)

11.648437906890807


##Audio 

In [ ]:
path_dirty = '/content/drive/MyDrive/ENGR-E-533 /DATA/HW2/test_x_01.wav'
xt1, sr=librosa.load(path_dirty, sr=None) 
print("The sampling rate(sr) for clean is {}".format(sr))
Xt1 = librosa.stft(xt1,n_fft=1024, hop_length=512)

Xt1_abs = np.abs(Xt1).T
Xt1_abs=np.concatenate((np.random.rand(19,513)*1e-20,Xt1_abs))

image_Xt1 = np.zeros([Xt1_abs.shape[0]-19,20,Xt1_abs.shape[1]])
for i in range(Xt1_abs.shape[0]-19):
  image_Xt1[i,:,:]=Xt1_abs[i:i+20,:]
print(image_Xt1.shape)

image_Xt1 = np.expand_dims(image_Xt1, axis = 1)

path_dirty = '/content/drive/MyDrive/ENGR-E-533 /DATA/HW2/test_x_02.wav'
xt2, sr=librosa.load(path_dirty, sr=None) 
print("The sampling rate(sr) for clean is {}".format(sr))
Xt2 = librosa.stft(xt2,n_fft=1024, hop_length=512)

Xt2_abs = np.abs(Xt2).T
Xt2_abs=np.concatenate((np.random.rand(19,513)*1e-20,Xt2_abs))

image_Xt2 = np.zeros([Xt2_abs.shape[0]-19,20,Xt2_abs.shape[1]])
for i in range(Xt2_abs.shape[0]-19):
  image_Xt2[i,:,:]=Xt2_abs[i:i+20,:]
print(image_Xt2.shape)

image_Xt2 = np.expand_dims(image_Xt2, axis = 1)

print('Xt1_abs shape: {}'.format(Xt1_abs.shape))
print('Xt2_abs shape: {}'.format(Xt2_abs.shape))





The sampling rate(sr) for clean is 16000
(142, 20, 513)
The sampling rate(sr) for clean is 16000
(380, 20, 513)
Xt1_abs shape: (161, 513)
Xt2_abs shape: (399, 513)


In [ ]:
St1_pred = model.predict(image_Xt1)
St2_pred = model.predict(image_Xt2)

12/12 [==============================] - 0s 9ms/step


In [ ]:
Sh_t1 = (Xt1/np.abs(Xt1))*np.abs(St1_pred.T)
Sh_t2 = (Xt2/np.abs(Xt2))*np.abs(St2_pred.T)

In [ ]:
path_op1= '/content/drive/MyDrive/ENGR-E-533 /DATA/HW2/test_recons5.wav'
s_test1 = istft(Sh_t1, hop_length=512)
sf.write(path_op1, s_test1, sr)

path_op2 = '/content/drive/MyDrive/ENGR-E-533 /DATA/HW2/test_recons6.wav'
s_test2 = istft(Sh_t2, hop_length=512)
sf.write(path_op2, s_test2, sr)

##Noisy

In [ ]:
Audio(data='/content/drive/MyDrive/ENGR-E-533 /DATA/HW2/test_x_01.wav', rate=sr)

##Clean

In [ ]:
Audio(data='/content/drive/MyDrive/ENGR-E-533 /DATA/HW2/test_recons5.wav', rate=sr)

##Noisy

In [ ]:
Audio(data='/content/drive/MyDrive/ENGR-E-533 /DATA/HW2/test_x_02.wav', rate=sr)

##Clean

In [ ]:
Audio(data='/content/drive/MyDrive/ENGR-E-533 /DATA/HW2/test_recons6.wav', rate=sr)

In [ ]:
%%shell
jupyter nbconvert --to html /content/Deep_Learning_HW2_Q3.ipynb

[NbConvertApp] Converting notebook /content/Deep_Learning_HW2_Q3.ipynb to html
[NbConvertApp] Writing 1762339 bytes to /content/Deep_Learning_HW2_Q3.html
